<a href="https://colab.research.google.com/github/Khadiza13/DravidianLangTech-NAACL-Misogyny-/blob/main/Visual_only_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
TRAIN_DATA_PATH = '/kaggle/input/train-dataset/train/train.csv'
EVAL_DATA_PATH = '/kaggle/input/eval-dataset/dev/dev.csv'
TEST_DATA_PATH = '/kaggle/input/test-dataset/test/test.csv'
TEST_LABELS_PATH = '/kaggle/input/test-with-labels/test_with_labels/test_with_labels.csv'
# Load the training data
train_data = pd.read_csv(TRAIN_DATA_PATH)
eval_data = pd.read_csv(EVAL_DATA_PATH)
test_data = pd.read_csv(TEST_DATA_PATH)
test_labels=pd.read_csv(TEST_LABELS_PATH)

In [ ]:
train_label_counts = train_data['labels'].value_counts()
eval_label_counts = eval_data['labels'].value_counts()
test_label_counts=test_labels['labels'].value_counts()

# Display the counts
print("Training Data Label Distribution:")
print(train_label_counts)

print("\nValidation Data Label Distribution:")
print(eval_label_counts)

print("\nTest Data Label Distribution:")
print(test_label_counts)

Training Data Label Distribution:
labels
0    381
1    259
Name: count, dtype: int64

Validation Data Label Distribution:
labels
0    97
1    63
Name: count, dtype: int64

Test Data Label Distribution:
labels
0    122
1     78
Name: count, dtype: int64


In [ ]:
train_data.shape

(640, 3)

In [ ]:
eval_data.shape

(160, 3)

In [ ]:
test_data.shape

(200, 2)

In [ ]:
train_data.head()

,image_id,labels,transcriptions
0,888,0,\nഈ ചാടി ഓടി നടക്കണ മനുഷ്യനാണോടാ നിങ്ങളിത്രേം ...
1,554,1,മലയാള സിനിമയുടെ ഭാവി വടറാണി ഇവൾ തന്നെ നല്ല കുഴ...
2,556,1,ഒന്ന് പെറ്റത് ആണെങ്കിലും .. മുലയും വയറും ചാടിയ...
3,484,1,ഓൺലൈൻ പരിചയപ്പെട്ടവനെ കളി തരാമെന്ന് പറഞ്ഞു അപ്...
4,370,0,കാമുകിയും അൺലിമിറ്റഡ് നെറ്റ് ഓഫറുംഉള്ള പയ്യന്റ...


In [ ]:
test_labels.head()

,image_id,labels,transcriptions
0,954,0,ഇതിന് മാത്രം പിള്ളാരെ ഇവർക്കെങ്ങനെ cilşma??....
1,239,0,ഇത് ഞാൻ ചെറുതായിരുന്നപ്പോൾ ഇത് 5ആം ക്ലാസ്സ് വര...
2,61,1,എന്തൊക്കെ ആകിയിട്ടും ഒരു മെന വരുന്നില്ലലോ നിഖി...
3,984,0,ഇഞ്ചുറിക്കു ശേഷം ഇന്റർനാഷണൽ ഫുട്ബോൾലേക്കു തിരി...
4,774,0,* ഡോക്ടർ\nകണ്ണൻ സഹാവ്\nയ്യോ.. ഒരു equipment ഉള...


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import os
import shutil


# Helper function to organize data based on the CSV file
def organize_data_from_csv(csv_path, images_dir, target_dir):
    """
    Organizes images into class-labeled directories based on a CSV file.

    Parameters:
        csv_path (str): Path to the CSV file containing image IDs and labels.
        images_dir (str): Path to the directory containing images.
        target_dir (str): Target directory where images will be organized.
    """
    data = pd.read_csv(csv_path)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    for _, row in data.iterrows():
        image_filename = f"{row['image_id']}.jpg"  # Match image filenames
        label = str(row['labels'])  # Convert label to string for folder names
        source_path = os.path.join(images_dir, image_filename)
        class_dir = os.path.join(target_dir, label)
        os.makedirs(class_dir, exist_ok=True)
        if os.path.exists(source_path):
            shutil.copy(source_path, class_dir)

# Paths for your dataset
TRAIN_CSV_PATH = '/kaggle/input/train-dataset/train/train.csv'
TRAIN_IMAGES_DIR = '/kaggle/input/train-dataset/train/'
EVAL_CSV_PATH = '/kaggle/input/eval-dataset/dev/dev.csv'
EVAL_IMAGES_DIR = '/kaggle/input/eval-dataset/dev/'
TEST_CSV_PATH = '/kaggle/input/test-dataset/test/test.csv'
TEST_IMAGES_DIR = '/kaggle/input/test-dataset/test/'
TEST_LABELS_IMAGES_DIR = '/kaggle/input/test-with-labels/test_with_labels/'
TEST_LABELS_PATH = '/kaggle/input/test-with-labels/test_with_labels/test_with_labels.csv'

MERGED_DIR = '/kaggle/working/merged_train_eval'  # Merged train/eval directory
TEST_DIR = '/kaggle/working/test_data'  # Test data directory

# Organize train and evaluation data into the merged directory
organize_data_from_csv(TRAIN_CSV_PATH, TRAIN_IMAGES_DIR, MERGED_DIR)
organize_data_from_csv(EVAL_CSV_PATH, EVAL_IMAGES_DIR, MERGED_DIR)

# Organize test data
organize_data_from_csv(TEST_LABELS_PATH, TEST_LABELS_IMAGES_DIR, TEST_DIR)


In [ ]:
# Prepare data generators
data_gen = ImageDataGenerator(rescale=1.0/255)

train_eval_generator = data_gen.flow_from_directory(
    directory=MERGED_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
)

test_generator = data_gen.flow_from_directory(
    directory=TEST_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False,  # Ensure order matches for predictions and labels.
)

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
# Define the models
def build_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Train and evaluate models
models = {
    "CNN": build_cnn(),
    # "Vision Transformer": build_vit(),
    # "Swin Transformer": build_swin()
}

results = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(train_eval_generator, epochs=5, verbose=1)
    print(f"Evaluating {model_name} on test data...")
    predictions = model.predict(test_generator)
    predicted_labels = (predictions > 0.5).astype(int).flatten()
    true_labels = test_generator.classes

    # Classification report
    report = classification_report(true_labels, predicted_labels, target_names=["Non-Misogyny", "Misogyny"])
    results[model_name] = report

# Print classification reports
for model_name, report in results.items():
    print(f"\nClassification Report for {model_name}:\n")
    print(report)



Training CNN...
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 274ms/step - accuracy: 0.5394 - loss: 3.2314
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7529 - loss: 0.5388
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.7803 - loss: 0.4900
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 307ms/step - accuracy: 0.8213 - loss: 0.4069
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.9103 - loss: 0.2341
Evaluating CNN on test data...
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 486ms/step

Classification Report for CNN:

              precision    recall  f1-score   support

Non-Misogyny       0.72      0.80      0.76       122
    Misogyny       0.62      0.51      0.56        78

    accuracy                           0.69       200
   macro avg       0.67      0.66      0.66       200
weighted avg       0.68      0.69      0.68       200

